In [11]:
from glob import glob
import pandas as pd
import os
import nbformat
import probml_utils.url_utils as url_utils

help(url_utils)

Help on module probml_utils.url_utils in probml_utils:

NAME
    probml_utils.url_utils

FUNCTIONS
    check_dead_urls(urls: Any, print_dead_url=True)
        returns if urls are dead or not
    
    colab_to_githubraw_url(url)
        convert colab .ipynb url to github raw .ipynb url
    
    colab_url_to_github_url(url)
        convert colab .ipynb url to github .ipynb url
    
    create_firestore_db(key_path)
    
    dict_to_csv(key_value_dict, csv_name, columns=['key', 'url'])
    
    extract_scripts_name_from_caption(caption)
        extract foo.py from ...{https//:<path/to/>foo.py}{foo.py}...
        Input: caption
        Output: ['foo.py']
    
    figure_url_mapping_from_lof(lof_file_path, csv_name, convert_to_which_url='colab', base_url='https://github.com/probml/pyprobml/blob/master/notebooks', book_no=1)
    
    github_to_rawcontent_url(github_url)
    
    github_url_to_colab_url(url)
        convert github .ipynb url to colab .ipynb url
    
    is_dead_url(link)
    

In [12]:
root_path = "../notebooks"


def get_notebook_path(book_str, chap_no, nb_name):
    return os.path.join(root_path, book_str, chap_no, nb_name)


def seperate_stuffs(nb_path):
    return nb_path.split("/")[-3:]

In [13]:
book2 = glob("../notebooks/book2/*/*.ipynb") + glob("../notebooks/book1/*/*.ipynb")
book2
len(book2)

478

In [14]:
nb_list = list(map(seperate_stuffs, book2))
df_nb_list = pd.DataFrame(nb_list, columns=["book_no", "chap_no", "nb_name"])
df_nb_list

,book_no,chap_no,nb_name
0,book2,18,gp_deep_kernel_learning.ipynb
1,book2,18,gpr_demo_ard.ipynb
2,book2,18,gp_spatial_demo.ipynb
3,book2,18,krr_vs_gpr.ipynb
4,book2,18,gp_kernel_opt.ipynb
...,...,...,...
473,book1,08,lrschedule_tf.ipynb
474,book1,08,steepestDescentDemo.ipynb
475,book1,08,newtonsMethodMinQuad.ipynb
476,book1,22,matrix_factorization_recommender_surprise_lib....


In [15]:
df_nb_list_grp = df_nb_list.groupby("nb_name").agg(lambda x: list(x)).reset_index()
df_nb_list_grp

,nb_name,book_no,chap_no
0,KLfwdReverseMixGauss.ipynb,[book1],[06]
1,LinearRegressionProbML.ipynb,[book1],[11]
2,MIC_correlation_2d.ipynb,[book1],[06]
3,ab_test_demo.ipynb,[book2],[34]
4,activation_fun_deriv.ipynb,[book1],[13]
...,...,...,...
459,word_analogies_jax.ipynb,[book1],[20]
460,word_analogies_torch.ipynb,[book1],[20]
461,xor_heaviside.ipynb,[book1],[13]
462,yeast_data_viz.ipynb,[book1],[21]


In [16]:
def is_query_in_nb(notebook, query):
    """
    fun should take one argument: code
    """
    nb = nbformat.read(notebook, as_version=4)
    for cell in nb.cells:
        code = cell["source"]
        if query in code:
            return 1
    return 0


def get_n_cells_nb(notebook):
    """
    fun should take one argument: code
    """
    nb = nbformat.read(notebook, as_version=4)
    return len(nb.cells)


def get_original_nb(df_nb_list_grp_ser):
    nb_name = df_nb_list_grp_ser["nb_name"]
    books = df_nb_list_grp_ser["book_no"]
    chaps = df_nb_list_grp_ser["chap_no"]
    t = []
    for book, chap in zip(books, chaps):
        nb_path = get_notebook_path(book, chap, nb_name)
        is_source = is_query_in_nb(nb_path, "Source of this notebook")
        t.append(is_source)
    return t

In [17]:
df_nb_list_grp["is_source_present"] = df_nb_list_grp.apply(get_original_nb, axis=1)
df_nb_list_grp

,nb_name,book_no,chap_no,is_source_present
0,KLfwdReverseMixGauss.ipynb,[book1],[06],[0]
1,LinearRegressionProbML.ipynb,[book1],[11],[0]
2,MIC_correlation_2d.ipynb,[book1],[06],[0]
3,ab_test_demo.ipynb,[book2],[34],[0]
4,activation_fun_deriv.ipynb,[book1],[13],[0]
...,...,...,...,...
459,word_analogies_jax.ipynb,[book1],[20],[0]
460,word_analogies_torch.ipynb,[book1],[20],[0]
461,xor_heaviside.ipynb,[book1],[13],[0]
462,yeast_data_viz.ipynb,[book1],[21],[0]


In [45]:
# Delete duplicate notebooks from book2
def del_duplicate_notebook(df_root_ser):
    is_source = df_root_ser["is_source_present"]
    nb_name = df_root_ser["nb_name"]
    # print(is_source)
    for i in range(len(is_source)):
        if is_source[i] == 1 and df_root_ser["book_no"][i] == "book2":  # delete only book2's duplicate notebook:
            nb_path = get_notebook_path(df_root_ser["book_no"][i], df_root_ser["chap_no"][i], nb_name)
            if is_query_in_nb(nb_path, "/pyprobml/"):  # delete notebooks who have pyprobml links
                os.remove(nb_path)
                print("Deleted: ", nb_path)


def del_fig_notebook(df_root_ser):
    is_source = df_root_ser["is_source_present"]
    nb_name = df_root_ser["nb_name"]
    # print(is_source)

    if "fig_" in nb_name and df_root_ser["book_no"] == "book2":  # delete only book2's notebook:
        nb_path = get_notebook_path(df_root_ser["book_no"], df_root_ser["chap_no"], nb_name)
        os.remove(nb_path)
        print("Deleted: ", nb_path)


df_root = df_nb_list_grp
# df_nb_list_grp.apply(del_duplicate_notebook, axis=1)
df_nb_list_grp.apply(del_fig_notebook, axis=1)

Deleted:  ../notebooks/book2/11/fig_10_5.ipynb
Deleted:  ../notebooks/book2/12/fig_11_20.ipynb
Deleted:  ../notebooks/book2/15/fig_14_5.ipynb
Deleted:  ../notebooks/book2/29/fig_28_11.ipynb
Deleted:  ../notebooks/book2/02/fig_2_5.ipynb
Deleted:  ../notebooks/book2/03/fig_3_2.ipynb


0      None
1      None
2      None
3      None
4      None
       ... 
459    None
460    None
461    None
462    None
463    None
Length: 464, dtype: object

In [19]:
def get_root_col(df_root_ser, col):
    is_source = df_root_ser["is_source_present"]
    nb_name = df_root_ser["nb_name"]

    if is_source.count(0) == 0:
        print(f"{nb_name} is not in pyprobml!")
        return df_root_ser[col][0]

    elif is_source.count(0) > 1:
        print(f"{nb_name} - multiple copies exist")

    else:
        return df_root_ser[col][is_source.index(0)]


df_root = df_nb_list_grp
df_root["chap_no"] = df_nb_list_grp.apply(get_root_col, col="chap_no", axis=1)
df_root["book_no"] = df_nb_list_grp.apply(get_root_col, col="book_no", axis=1)
df_root

bootstrap_filter.ipynb is not in pyprobml!
bootstrap_filter_maneuver.ipynb is not in pyprobml!
ekf_mlp.ipynb is not in pyprobml!
ekf_vs_ukf.ipynb is not in pyprobml!
gauss-bp-1d-line.ipynb is not in pyprobml!
gprDemoArd.ipynb is not in pyprobml!
gprDemoChangeHparams.ipynb is not in pyprobml!
gprDemoNoiseFree.ipynb is not in pyprobml!
pendulum_1d.ipynb is not in pyprobml!
rbpf_maneuver.ipynb is not in pyprobml!
rbpf_maneuver_demo.ipynb is not in pyprobml!
sis_vs_smc.ipynb is not in pyprobml!
bootstrap_filter.ipynb is not in pyprobml!
bootstrap_filter_maneuver.ipynb is not in pyprobml!
ekf_mlp.ipynb is not in pyprobml!
ekf_vs_ukf.ipynb is not in pyprobml!
gauss-bp-1d-line.ipynb is not in pyprobml!
gprDemoArd.ipynb is not in pyprobml!
gprDemoChangeHparams.ipynb is not in pyprobml!
gprDemoNoiseFree.ipynb is not in pyprobml!
pendulum_1d.ipynb is not in pyprobml!
rbpf_maneuver.ipynb is not in pyprobml!
rbpf_maneuver_demo.ipynb is not in pyprobml!
sis_vs_smc.ipynb is not in pyprobml!


,nb_name,book_no,chap_no,is_source_present
0,KLfwdReverseMixGauss.ipynb,book1,06,[0]
1,LinearRegressionProbML.ipynb,book1,11,[0]
2,MIC_correlation_2d.ipynb,book1,06,[0]
3,ab_test_demo.ipynb,book2,34,[0]
4,activation_fun_deriv.ipynb,book1,13,[0]
...,...,...,...,...
459,word_analogies_jax.ipynb,book1,20,[0]
460,word_analogies_torch.ipynb,book1,20,[0]
461,xor_heaviside.ipynb,book1,13,[0]
462,yeast_data_viz.ipynb,book1,21,[0]


In [20]:
df_root[df_root["book_no"].isna()]

,nb_name,book_no,chap_no,is_source_present


In [21]:
df_root

,nb_name,book_no,chap_no,is_source_present
0,KLfwdReverseMixGauss.ipynb,book1,06,[0]
1,LinearRegressionProbML.ipynb,book1,11,[0]
2,MIC_correlation_2d.ipynb,book1,06,[0]
3,ab_test_demo.ipynb,book2,34,[0]
4,activation_fun_deriv.ipynb,book1,13,[0]
...,...,...,...,...
459,word_analogies_jax.ipynb,book1,20,[0]
460,word_analogies_torch.ipynb,book1,20,[0]
461,xor_heaviside.ipynb,book1,13,[0]
462,yeast_data_viz.ipynb,book1,21,[0]


In [14]:
url_utils.github_to_rawcontent_url("https://github.com/probml/pyprobml/blob/master/notebooks/book2/03/dtheory.ipynb")

'https://raw.githubusercontent.com/probml/pyprobml/master/notebooks/book2/03/dtheory.ipynb'

In [22]:
# check dead urls
# df_root["url"] = df_root.apply(
#     lambda x: url_utils.make_url_from_chapter_no_and_script_name(
#         chapter_no=int(x["chap_no"]),
#         script_name=x["nb_name"],
#         book_no=int(x["book_no"][-1]),
#         convert_to_which_url="github-raw",
#     ),
#     axis=1,
# )
# df_root

In [23]:
# url_utils.check_dead_urls(list(df_root["url"]))

In [17]:
# Add colab url
df_root["colab_url"] = df_root.apply(
    lambda x: url_utils.make_url_from_chapter_no_and_script_name(
        chapter_no=int(x["chap_no"]),
        script_name=x["nb_name"],
        book_no=int(x["book_no"][-1]),
        convert_to_which_url="colab",
    ),
    axis=1,
)

# Add colab url
df_root["github_url"] = df_root.apply(
    lambda x: url_utils.make_url_from_chapter_no_and_script_name(
        chapter_no=int(x["chap_no"]),
        script_name=x["nb_name"],
        book_no=int(x["book_no"][-1]),
        convert_to_which_url="github",
    ),
    axis=1,
)
df_root

,nb_name,book_no,chap_no,is_source_present,url,colab_url,github_url
0,KLfwdReverseMixGauss.ipynb,book1,06,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
1,LinearRegressionProbML.ipynb,book1,11,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
2,MIC_correlation_2d.ipynb,book1,06,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
3,ab_test_demo.ipynb,book2,34,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
4,activation_fun_deriv.ipynb,book2,16,[1],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
...,...,...,...,...,...,...,...
454,word_analogies_jax.ipynb,book1,20,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
455,word_analogies_torch.ipynb,book1,20,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
456,xor_heaviside.ipynb,book1,13,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
457,yeast_data_viz.ipynb,book1,21,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...


In [18]:
t = df_root["url"][1]
t

'https://raw.githubusercontent.com/probml/pyprobml/master/notebooks/book1/11/LinearRegressionProbML.ipynb'

In [19]:
enclose_span = lambda text, nb_id: f"<span id={nb_id}>{text}</span>"
to_md_url = lambda text, url: f"[{text}]({url})"

# to_md_url(enclose_span("GAN_JAX_CelebA_demo.ipynb"), t)
df_root["md_colab_url"] = df_root.apply(
    lambda x: to_md_url(enclose_span("colab", x["nb_name"]), x["colab_url"]), axis=1
)
df_root["md_github_url"] = df_root.apply(
    lambda x: to_md_url(enclose_span("github", x["nb_name"]), x["github_url"]), axis=1
)
df_root

,nb_name,book_no,chap_no,is_source_present,url,colab_url,github_url,md_colab_url,md_github_url
0,KLfwdReverseMixGauss.ipynb,book1,06,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=KLfwdReverseMixGauss.ipynb>colab</sp...,[<span id=KLfwdReverseMixGauss.ipynb>github</s...
1,LinearRegressionProbML.ipynb,book1,11,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=LinearRegressionProbML.ipynb>colab</...,[<span id=LinearRegressionProbML.ipynb>github<...
2,MIC_correlation_2d.ipynb,book1,06,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=MIC_correlation_2d.ipynb>colab</span...,[<span id=MIC_correlation_2d.ipynb>github</spa...
3,ab_test_demo.ipynb,book2,34,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=ab_test_demo.ipynb>colab</span>](htt...,[<span id=ab_test_demo.ipynb>github</span>](ht...
4,activation_fun_deriv.ipynb,book2,16,[1],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=activation_fun_deriv.ipynb>colab</sp...,[<span id=activation_fun_deriv.ipynb>github</s...
...,...,...,...,...,...,...,...,...,...
454,word_analogies_jax.ipynb,book1,20,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=word_analogies_jax.ipynb>colab</span...,[<span id=word_analogies_jax.ipynb>github</spa...
455,word_analogies_torch.ipynb,book1,20,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=word_analogies_torch.ipynb>colab</sp...,[<span id=word_analogies_torch.ipynb>github</s...
456,xor_heaviside.ipynb,book1,13,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=xor_heaviside.ipynb>colab</span>](ht...,[<span id=xor_heaviside.ipynb>github</span>](h...
457,yeast_data_viz.ipynb,book1,21,[0],https://raw.githubusercontent.com/probml/pypro...,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...,[<span id=yeast_data_viz.ipynb>colab</span>](h...,[<span id=yeast_data_viz.ipynb>github</span>](...


In [20]:
df_final = df_root[["nb_name", "md_colab_url", "md_github_url"]]
df_final.columns = ["Notebook", "Colab url", "Github url"]
df_final

,Notebook,Colab url,Github url
0,KLfwdReverseMixGauss.ipynb,[<span id=KLfwdReverseMixGauss.ipynb>colab</sp...,[<span id=KLfwdReverseMixGauss.ipynb>github</s...
1,LinearRegressionProbML.ipynb,[<span id=LinearRegressionProbML.ipynb>colab</...,[<span id=LinearRegressionProbML.ipynb>github<...
2,MIC_correlation_2d.ipynb,[<span id=MIC_correlation_2d.ipynb>colab</span...,[<span id=MIC_correlation_2d.ipynb>github</spa...
3,ab_test_demo.ipynb,[<span id=ab_test_demo.ipynb>colab</span>](htt...,[<span id=ab_test_demo.ipynb>github</span>](ht...
4,activation_fun_deriv.ipynb,[<span id=activation_fun_deriv.ipynb>colab</sp...,[<span id=activation_fun_deriv.ipynb>github</s...
...,...,...,...
454,word_analogies_jax.ipynb,[<span id=word_analogies_jax.ipynb>colab</span...,[<span id=word_analogies_jax.ipynb>github</spa...
455,word_analogies_torch.ipynb,[<span id=word_analogies_torch.ipynb>colab</sp...,[<span id=word_analogies_torch.ipynb>github</s...
456,xor_heaviside.ipynb,[<span id=xor_heaviside.ipynb>colab</span>](ht...,[<span id=xor_heaviside.ipynb>github</span>](h...
457,yeast_data_viz.ipynb,[<span id=yeast_data_viz.ipynb>colab</span>](h...,[<span id=yeast_data_viz.ipynb>github</span>](...


In [42]:
df_final.iloc[0, 1]

'[<span id=KLfwdReverseMixGauss.ipynb>colab</span>](https://colab.research.google.com/github/probml/pyprobml/blob/master/notebooks/book1/06/KLfwdReverseMixGauss.ipynb)'

## Handle supplementary notebooks

In [36]:
github_root = "https://github.com/probml/pyprobml/blob/master"
colab_root = "https://colab.research.google.com/github/probml/pyprobml/blob/master"

In [37]:
supp_book2 = glob("../notebooks/book2/*/*/*.ipynb") + glob("../notebooks/book1/*/*/*.ipynb")
supp_book2
len(supp_book2), supp_book2[0]

(17, '../notebooks/book2/09/supplementary/discretized_ssm.ipynb')

In [43]:
nb_github_colab_list = list(
    map(lambda x: [x.split("/")[-1], colab_root + x.replace("..", ""), github_root + x.replace("..", "")], supp_book2)
)
df_supp = pd.DataFrame(nb_github_colab_list, columns=df_final.columns)
df_supp

,Notebook,Colab url,Github url
0,discretized_ssm.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
1,hmm_bach_chorales_distrax.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
2,hmm_with_ngram.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
3,hmm_flows_pos_tagging.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
4,hmm_poisson_changepoint_tfp.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
5,hmm_poisson_changepoint_jax.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
6,hmm_lillypad.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
7,Tabular_SARSA.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
8,rl_demos_tf.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...
9,randomized_priors_enn.ipynb,https://colab.research.google.com/github/probm...,https://github.com/probml/pyprobml/blob/master...


In [44]:
df_supp["Colab url"] = df_supp.apply(lambda x: to_md_url(enclose_span("colab", x["Notebook"]), x["Colab url"]), axis=1)
df_supp["Github url"] = df_supp.apply(
    lambda x: to_md_url(enclose_span("github", x["Notebook"]), x["Github url"]), axis=1
)
df_supp

,Notebook,Colab url,Github url
0,discretized_ssm.ipynb,[<span id=discretized_ssm.ipynb>colab</span>](...,[<span id=discretized_ssm.ipynb>github</span>]...
1,hmm_bach_chorales_distrax.ipynb,[<span id=hmm_bach_chorales_distrax.ipynb>cola...,[<span id=hmm_bach_chorales_distrax.ipynb>gith...
2,hmm_with_ngram.ipynb,[<span id=hmm_with_ngram.ipynb>colab</span>](h...,[<span id=hmm_with_ngram.ipynb>github</span>](...
3,hmm_flows_pos_tagging.ipynb,[<span id=hmm_flows_pos_tagging.ipynb>colab</s...,[<span id=hmm_flows_pos_tagging.ipynb>github</...
4,hmm_poisson_changepoint_tfp.ipynb,[<span id=hmm_poisson_changepoint_tfp.ipynb>co...,[<span id=hmm_poisson_changepoint_tfp.ipynb>gi...
5,hmm_poisson_changepoint_jax.ipynb,[<span id=hmm_poisson_changepoint_jax.ipynb>co...,[<span id=hmm_poisson_changepoint_jax.ipynb>gi...
6,hmm_lillypad.ipynb,[<span id=hmm_lillypad.ipynb>colab</span>](htt...,[<span id=hmm_lillypad.ipynb>github</span>](ht...
7,Tabular_SARSA.ipynb,[<span id=Tabular_SARSA.ipynb>colab</span>](ht...,[<span id=Tabular_SARSA.ipynb>github</span>](h...
8,rl_demos_tf.ipynb,[<span id=rl_demos_tf.ipynb>colab</span>](http...,[<span id=rl_demos_tf.ipynb>github</span>](htt...
9,randomized_priors_enn.ipynb,[<span id=randomized_priors_enn.ipynb>colab</s...,[<span id=randomized_priors_enn.ipynb>github</...


In [45]:
df_supp.iloc[0, 2]

'[<span id=discretized_ssm.ipynb>github</span>](https://github.com/probml/pyprobml/blob/master/notebooks/book2/09/supplementary/discretized_ssm.ipynb)'

## Combine supplementary + chapters notebooks

In [46]:
df_chap_supp = pd.concat([df_final, df_supp])
df_chap_supp = df_chap_supp.sort_values(by="Notebook", key=lambda col: col.str.lower())
df_chap_supp

,Notebook,Colab url,Github url
3,ab_test_demo.ipynb,[<span id=ab_test_demo.ipynb>colab</span>](htt...,[<span id=ab_test_demo.ipynb>github</span>](ht...
4,activation_fun_deriv.ipynb,[<span id=activation_fun_deriv.ipynb>colab</sp...,[<span id=activation_fun_deriv.ipynb>github</s...
5,activation_fun_deriv_jax.ipynb,[<span id=activation_fun_deriv_jax.ipynb>colab...,[<span id=activation_fun_deriv_jax.ipynb>githu...
6,activation_fun_plot.ipynb,[<span id=activation_fun_plot.ipynb>colab</spa...,[<span id=activation_fun_plot.ipynb>github</sp...
7,adf_logistic_regression_demo.ipynb,[<span id=adf_logistic_regression_demo.ipynb>c...,[<span id=adf_logistic_regression_demo.ipynb>g...
...,...,...,...
454,word_analogies_jax.ipynb,[<span id=word_analogies_jax.ipynb>colab</span...,[<span id=word_analogies_jax.ipynb>github</spa...
455,word_analogies_torch.ipynb,[<span id=word_analogies_torch.ipynb>colab</sp...,[<span id=word_analogies_torch.ipynb>github</s...
456,xor_heaviside.ipynb,[<span id=xor_heaviside.ipynb>colab</span>](ht...,[<span id=xor_heaviside.ipynb>github</span>](h...
457,yeast_data_viz.ipynb,[<span id=yeast_data_viz.ipynb>colab</span>](h...,[<span id=yeast_data_viz.ipynb>github</span>](...


In [47]:
df_chap_supp.to_markdown("../notebooks.md", index=False)

In [ ]:
pd.concat()

In [25]:
df_external = pd.read_csv("external_links.csv")
df_external

,key,type,url
0,hmm_casino_training.ipynb,github,https://github.com/probml/ssm-jax/blob/main/ss...
1,hmm_casino_training_old.ipynb,github,https://github.com/probml/pyprobml/blob/master...


In [27]:
df1 = pd.DataFrame([[0, 1], [2, 3]])
df1

,0,1
0,0,1
1,2,3


In [31]:
df2 = pd.DataFrame([[11, 12], [13, 14]], index=[1, 2])
df2

,0,1
1,11,12
2,13,14


In [38]:
import numpy as np

array([1])

In [41]:
df1.drop(index=np.intersect1d(df1.index.values, df2.index.values))

,0,1
0,0,1
